# New split Charades-STA
Charades does not have a proper validation split to tune HPS on. <br>
In this script we fuse train and test and derive, by random split, train_dev and val_dev.

In [1]:
import json
import random
import subprocess
from datetime import datetime

In [2]:
#Load current splits:
train = json.load(open('../data/processed/charades-sta/train-01.json','r'))
test  = json.load(open('../data/processed/charades-sta/test-01.json','r'))
print(train.keys())
print(train['git_hash'])
print(train['responsible'])
print(train['date'])

dict_keys(['videos', 'moments', 'date', 'git_hash', 'responsible'])
6913c98bc58ef1d3449bf3dca249069c0496236b
EscorciaSSGR
2019-01-21T02:00:48.670279


In [3]:
# Create common info
date = str(datetime.now())
git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'],universal_newlines=True).strip()
responsible = 'Mattia Soldan'

In [4]:
# Initialize dictionary
train_dev = {'date':date, 'git_hash':git_hash, 'responsible':responsible}
val_dev   = {'date':date, 'git_hash':git_hash, 'responsible':responsible}

In [5]:
# Merge videos:
all_videos_id    = set(list(train['videos'].keys())+list(test['videos'].keys()))
train_dev_perc   = 0.80
train_dev_videos = random.sample(all_videos_id, int(train_dev_perc * len(all_videos_id)))
val_dev_videos   = all_videos_id.difference(set(train_dev_videos))

In [6]:
# Analize overlapping with previous splits:
l_v_tr = len(train['videos'].keys())
l_v_te = len(test['videos'].keys())
print(f'Number of videos in train (original split) {l_v_tr}')
print(f'Number of videos in test (original split) {l_v_te}')

l_m_tr = len(train['moments'])
l_m_te = len(test['moments'])
print(f'Number of moments in train (original split) {l_m_tr}')
print(f'Number of moments in test (original split) {l_m_te}')

inter_tr = set(train_dev_videos).intersection(set(train['videos'].keys()))
inter_te = set(val_dev_videos).intersection(set(test['videos'].keys()))
print(f'Num overlapping video in training: {len(inter_tr)}')
print(f'Num overlapping video in testing: {len(inter_te)}')

print(f'Overlapping % video train {len(inter_tr)/l_v_tr *100:.2f}')
print(f'Overlapping % video test {len(inter_te)/l_v_te*100:.2f}')

Number of videos in train (original split) 5336
Number of videos in test (original split) 1334
Number of moments in train (original split) 12404
Number of moments in test (original split) 3720
Num overlapping video in training: 4254
Num overlapping video in testing: 252
Overlapping % video train 79.72
Overlapping % video test 18.89


In [7]:
# Split videos according to these video ids
tot_videos = train['videos']
tot_videos.update(test['videos'])
train_dev['videos'] = {k:v for k,v in tot_videos.items() if k in train_dev_videos}
val_dev['videos']   = {k:v for k,v in tot_videos.items() if k in val_dev_videos}
# Split moments
tot_moments = train['moments'] + test['moments']
train_dev['moments'] = [m for m in tot_moments if m['video'] in train_dev_videos]
val_dev['moments']   = [m for m in tot_moments if m['video'] in val_dev_videos]

In [8]:
# Analize new splits:
l_v_tr = len(train_dev['videos'].keys())
l_v_va = len(val_dev['videos'].keys())
print(f'Number of videos in train (original split) {l_v_tr}')
print(f'Number of videos in test (original split) {l_v_va}')

l_m_tr = len(train_dev['moments'])
l_m_va = len(val_dev['moments'])
print(f'Number of moments in train (original split) {l_m_tr}')
print(f'Number of moments in test (original split) {l_m_va}')

Number of videos in train (original split) 5336
Number of videos in test (original split) 1334
Number of moments in train (original split) 12901
Number of moments in test (original split) 3223


In [9]:
# Dump data
json.dump(train_dev, open('../data/processed/charades-sta/train-dev.json','w'))
json.dump(val_dev,   open('../data/processed/charades-sta/val-dev.json','w'))

In [1]:
import json
import random
import subprocess
from datetime import datetime

In [2]:
#Load current splits:
train = json.load(open('../data/processed/charades-sta/train-01.json','r'))
print(train.keys())
print(train['git_hash'])
print(train['responsible'])
print(train['date'])

dict_keys(['videos', 'moments', 'date', 'git_hash', 'responsible'])
6913c98bc58ef1d3449bf3dca249069c0496236b
EscorciaSSGR
2019-01-21T02:00:48.670279


In [3]:
# Create common info
date = str(datetime.now())
git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'],universal_newlines=True).strip()
responsible = 'Mattia Soldan'

In [4]:
# Initialize dictionary
train_dev2 = {'date':date, 'git_hash':git_hash, 'responsible':responsible}
val_dev2   = {'date':date, 'git_hash':git_hash, 'responsible':responsible}

In [7]:
# Merge videos:
all_videos_id    = set(list(train['videos'].keys()))
train_dev_perc   = 0.80
train_dev_videos = random.sample(all_videos_id, int(train_dev_perc * len(all_videos_id)))
val_dev_videos   = all_videos_id.difference(set(train_dev_videos))

In [8]:
# Split videos according to these video ids
tot_videos = train['videos']
train_dev2['videos'] = {k:v for k,v in tot_videos.items() if k in train_dev_videos}
val_dev2['videos']   = {k:v for k,v in tot_videos.items() if k in val_dev_videos}
# Split moments
tot_moments = train['moments'] 
train_dev2['moments'] = [m for m in tot_moments if m['video'] in train_dev_videos]
val_dev2['moments']   = [m for m in tot_moments if m['video'] in val_dev_videos]

In [11]:
# Analize new splits:
n = len(train['videos'].keys())
print(f'Number of videos in train (original split) {n}')
n = len(train_dev2['videos'].keys())
print(f'Number of videos in train_dev2 {n}')
n = len(val_dev2['videos'].keys())
print(f'Number of videos in val_dev2 {n}')

n = len(train['moments'])
print(f'Number of moments in train (original) {n}')
n = len(train_dev2['moments'])
print(f'Number of moments in train {n}')
n = len(val_dev2['moments'])
print(f'Number of moments in test {n}')

Number of videos in train (original split) 5336
Number of videos in train_dev2 4268
Number of videos in val_dev2 1068
Number of moments in train (original) 12404
Number of moments in train 9894
Number of moments in test 2510


In [12]:
# Dump data
json.dump(train_dev2, open('../data/processed/charades-sta/train-dev2.json','w'))
json.dump(val_dev2,   open('../data/processed/charades-sta/val-dev2.json','w'))